In [1]:
import numpy as np
import pandas as pd
import os

ModuleNotFoundError: No module named 'numpy'

In [13]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/sms-spam-collection-dataset")

print("Path to dataset files:", path)
data_file = os.path.join(path,"spam.csv")
print("File exists:", os.path.exists(data_file))
print("Files in directory:")
for f in os.listdir(path):
    print(f)

Path to dataset files: /Users/mohdsafeenkhan/.cache/kagglehub/datasets/uciml/sms-spam-collection-dataset/versions/1
File exists: True
Files in directory:
spam.csv


In [17]:
df = pd.read_csv(data_file,encoding='latin-1')[['v1','v2']]
#df = pd.read_csv(data_file, sep='\t', header=None, names=["label", "message"])df
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [19]:
df.columns = ['label','message']
df['label'] = df['label'].map({'ham':0,'spam':1}) #panda series map

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

x_train , x_test , y_train,y_test = train_test_split(df['message'],df['label'],test_size = 0.2 , random_state=42)

vectorizer = TfidfVectorizer(stop_words = 'english')

x_train_vec = vectorizer.fit_transform(x_train)
x_test_vec = vectorizer.transform(x_test)
print(x_test_vec)
print(x_test_vec)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 7693 stored elements and shape (1115, 7472)>
  Coords	Values
  (0, 1756)	0.31111329907426943
  (0, 2679)	0.3500886226408095
  (0, 2974)	0.34299776014114036
  (0, 3239)	0.34299776014114036
  (0, 3457)	0.3500886226408095
  (0, 3752)	0.1718556592061185
  (0, 4543)	0.38197308370768035
  (0, 6816)	0.4006242977875035
  (0, 7229)	0.2947064107791228
  (1, 1934)	0.22392171769600464
  (1, 1970)	0.2461378627103295
  (1, 2651)	0.3269309971271071
  (1, 3716)	0.3178303138520559
  (1, 4760)	0.29866169283344046
  (1, 5532)	0.33866381848750327
  (1, 5738)	0.25559165628741076
  (1, 5739)	0.35520030142077386
  (1, 5744)	0.35520030142077386
  (1, 5812)	0.22078293973996208
  (1, 6604)	0.19484478334547534
  (1, 6607)	0.27039238853977376
  (2, 305)	0.3193869031809334
  (2, 2707)	0.4882288103453305
  (2, 3835)	0.4855870501823454
  (2, 4106)	0.5120683436791947
  :	:
  (1110, 1533)	0.2553888613819584
  (1110, 1599)	0.27571108916401915
  (1110, 1745)	

In [27]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report , confusion_matrix

model = MultinomialNB()
model.fit(x_train_vec,y_train)
model_pred = model.predict(x_test_vec)

print("Naive Bayes Results:")
print(confusion_matrix(y_test, model_pred))
print(classification_report(y_test, model_pred))


Naive Bayes Results:
[[965   0]
 [ 37 113]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       965
           1       1.00      0.75      0.86       150

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.96      1115



In [37]:
y_prob = model.predict_proba(x_test_vec)
#print(y_prob)
# Set a custom threshold
threshold = 0.16  # You can adjust this (e.g., 0.3 is more aggressive in flagging spam)

# Apply threshold to get final predictions
y_pred_custom = (y_prob[:, 1] >= threshold).astype(int)
print(f"Results for Threshold = {threshold}")
print(classification_report(y_test, y_pred_custom))
print(confusion_matrix(y_test, y_pred_custom))

Results for Threshold = 0.16
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       965
           1       0.91      0.91      0.91       150

    accuracy                           0.98      1115
   macro avg       0.95      0.95      0.95      1115
weighted avg       0.98      0.98      0.98      1115

[[951  14]
 [ 13 137]]
